In [184]:
"""
Скрипт для обработки данных резюме с сайта Работа в России
"""

'\nСкрипт для обработки данных резюме с сайта Работа в России\n'

In [185]:
import pandas as pd
import numpy as np
import openpyxl
import json
import ast
import re
import os
from datetime import datetime, timedelta
import time
from dateutil import parser
pd.options.mode.chained_assignment = None

In [186]:
def convert_date(cell):
    """
    Функция конвертации строки содержащей дату и время
    """

    try:
        temp_date = pd.to_datetime(cell,errors='coerce',utc=True,dayfirst=True)
       

        return temp_date
    except pd.errors.OutOfBoundsDatetime:
        return pd.NaT
    except ValueError:
        return pd.NaT



In [187]:
df = pd.read_excel('data/Часть 1.xlsx',usecols=['stateRegionCode','birthday','gender','dateCreate','dateModify',
                                                'academicDegree','worldskills','worldskillsInspectionStatus','abilympicsInspectionStatus',
                                                'abilympicsParticipation','volunteersInspectionStatus','volunteersParticipation',
                                               'driveLicenses','experience','professionsList','otherCertificates',
                                                'narkCertificate','narkInspectionStatus','educationList','additionalEducationList',
                                                'hardSkills','softSkills','workExperienceList','scheduleType','salary','busyType',
                                                'retrainingCapability','businessTrip','languageKnowledge',
                                                'relocation'])

In [188]:
df['birthday']

0      1970-04-20T00:00:00+0300
1      2006-08-03T16:00:00+0400
2      2005-05-11T16:00:00+0400
3      1987-08-17T16:00:00+0400
4      1977-12-20T15:00:00+0300
                 ...           
995    2007-04-27T16:00:00+0400
996    2007-06-02T16:00:00+0400
997    1983-02-21T15:00:00+0300
998    1989-02-14T15:00:00+0300
999    1998-10-04T16:00:00+0400
Name: birthday, Length: 1000, dtype: object

In [193]:
# Конвертируем в даты колонки с датами
df['birthday'] = df['birthday'].apply(convert_date)
df['dateCreate'] = df['dateCreate'].apply(convert_date)
df['dateModify'] = df['dateModify'].apply(convert_date)

In [194]:
df['dateModify']

0     2022-05-31 09:49:55+00:00
1     2023-06-30 10:01:04+00:00
2     2023-08-15 05:03:08+00:00
3     2020-04-15 13:14:42+00:00
4     2021-01-27 01:14:13+00:00
                 ...           
995   2023-06-15 04:49:29+00:00
996   2023-09-18 14:38:20+00:00
997   2022-11-18 20:10:13+00:00
998   2023-07-24 09:04:44+00:00
999   2023-06-08 07:29:19+00:00
Name: dateModify, Length: 1000, dtype: datetime64[ns, UTC]

In [191]:
# Создаем колонку дата рождения
df['Дата рождения'] = df['birthday'].dt.strftime('%d.%m.%Y')

In [195]:
# Получаем текущую дату
current_date = datetime.now()

In [196]:
current_date

datetime.datetime(2024, 3, 22, 15, 10, 13, 311061)

In [197]:
one_year_ago = current_date - timedelta(days=365)

In [200]:
one_year_ago.date()

datetime.date(2023, 3, 23)

In [202]:
pd.to_datetime(one_year_ago.date())

Timestamp('2023-03-23 00:00:00')

In [207]:
# Фильтруем год назад
df[df['dateModify'] >pd.to_datetime(one_year_ago,utc=True)].head()

,stateRegionCode,birthday,gender,dateCreate,dateModify,academicDegree,worldskills,worldskillsInspectionStatus,abilympicsInspectionStatus,abilympicsParticipation,...,softSkills,workExperienceList,scheduleType,salary,busyType,retrainingCapability,businessTrip,languageKnowledge,relocation,Дата рождения
1,7400000000000,2006-08-03 12:00:00+00:00,Мужской,2023-06-30 10:01:04+00:00,2023-06-30 10:01:04+00:00,NaN,[],Участие-в-чемпионате-Worldskills-не-подтверждено,Участие-в-движении-Абилимпикс-не-подтверждено,Не-принимал-участие-в-движении-Абилимпикс,...,[],[],Неполный-рабочий-день,18000,NaN,Не-готов-к-переобучению,Не-готов-к-командировкам,[],Не-готов-к-переезду,03.08.2006
2,3000000000000,2005-05-11 12:00:00+00:00,Мужской,2023-08-15 05:03:08+00:00,2023-08-15 05:03:08+00:00,NaN,[],Участие-в-чемпионате-Worldskills-не-подтверждено,Участие-в-движении-Абилимпикс-не-подтверждено,Не-принимал-участие-в-движении-Абилимпикс,...,[],[],Полный-рабочий-день,20000,NaN,Не-готов-к-переобучению,Готов-к-командировкам,"[{""codeLanguage"": ""Русский"", ""level"": ""Базовый...",Не-готов-к-переезду,11.05.2005
6,7400000000000,1975-11-24 12:00:00+00:00,Женский,2023-09-19 09:15:07+00:00,2023-09-20 03:27:42+00:00,NaN,[],Участие-в-чемпионате-Worldskills-не-подтверждено,Участие-в-движении-Абилимпикс-не-подтверждено,Не-принимал-участие-в-движении-Абилимпикс,...,[],"[{""achievements"": ""<p>Компьютерные курсы,владе...",Полный-рабочий-день,30000,NaN,Готов-к-переобучению,Не-готов-к-командировкам,[],Не-готов-к-переезду,24.11.1975
8,6300000000000,1969-03-06 12:00:00+00:00,Мужской,2023-09-05 07:51:30+00:00,2023-09-05 07:51:30+00:00,NaN,[],Участие-в-чемпионате-Worldskills-не-подтверждено,Участие-в-движении-Абилимпикс-не-подтверждено,Не-принимал-участие-в-движении-Абилимпикс,...,[],"[{""companyName"": ""оао волгоцеммаш"", ""dateFrom""...",Полный-рабочий-день,35000,NaN,Не-готов-к-переобучению,Не-готов-к-командировкам,[],Не-готов-к-переезду,06.03.1969
18,6300000000000,1976-12-23 12:00:00+00:00,Женский,2021-10-01 10:19:13+00:00,2023-09-29 08:34:36+00:00,NaN,[],Участие-в-чемпионате-Worldskills-не-подтверждено,Участие-в-движении-Абилимпикс-не-подтверждено,Не-принимал-участие-в-движении-Абилимпикс,...,[],"[{""companyName"": ""СВВАУЛ"", ""dateFrom"": ""1999-0...",Полный-рабочий-день,55000,NaN,Готов-к-переобучению,Готов-к-командировкам,[],Не-готов-к-переезду,23.12.1976
